In [1]:
import torch
from transformers import pipeline

# 1. Initialize the pipeline
summarizer = pipeline(
    task="text-generation",
    # model="google/gemma-3-1b-it",
    model="models/gemma-3-1b-sft/merged/",
    device=0,
    torch_dtype=torch.bfloat16
)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-26 03:43:41.350798: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Device set to use cuda:0


In [2]:
import json

input_file = "dataset/org_with_cefr_labels/ca_test_data_final_OFFICIAL.jsonl"
dataset = [json.loads(line.strip()) for line in open(input_file)]
print(dataset[0]['text'])
dataset = dataset[:100]

The people of the State of California do enact as follows:


SECTION 1.
The Legislature finds and declares all of the following:
(a) (1) Since 1899 congressionally chartered veterans’ organizations have provided a valuable service to our nation’s returning service members. These organizations help preserve the memories and incidents of the great hostilities fought by our nation, and preserve and strengthen comradeship among members.
(2) These veterans’ organizations also own and manage various properties including lodges, posts, and fraternal halls. These properties act as a safe haven where veterans of all ages and their families can gather together to find camaraderie and fellowship, share stories, and seek support from people who understand their unique experiences. This aids in the healing process for these returning veterans, and ensures their health and happiness.
(b) As a result of congressional chartering of these veterans’ organizations, the United States Internal Revenue Serv

In [3]:
import json
from jinja2 import Template

SYSTEM_PROMPT = "You are helpful assistant designed to make English legal text more readable for different target audience at different CEFR readability levels."

PROMPT_TEMPLATE = "Summarize the following text for a {{ level }} reader. {{ text }} \n Please output the summary as a paragraph."

jinja_template = Template(PROMPT_TEMPLATE)
print(jinja_template)

<Template memory:79998e58f730>


In [4]:
# from transformers import pipeline
# from tqdm import tqdm

# cefr_labels = ["A1", "A2", "B1", "B2", "C1", "C2"]

# for i in tqdm(range(len(dataset))):
    
#     text = dataset[i]["text"]
#     messages = [
#             [
#                 {
#                     "role": "system",
#                     "content": [{"type": "text", "text": prompt_json["system_prompt"]},]
#                 },
#                 {
#                     "role": "user",
#                     "content": [{"type": "text", "text": jinja_template.render(text=text, level=label)},]
#                 },
#             ]
#         for label in cefr_labels
#     ]

#     summaries = summarizer(messages, max_new_tokens=512, batch_size=6)

#     all_summaries = {}
#     for summary, label in zip(summaries, cefr_labels):
#         summary_text = summary[-1]["generated_text"][-1]["content"]

#         summary_text = summary_text.split("\n\n")
#         summary_text = "\n\n".join(summary_text[1:])

#         all_summaries[label] = summary_text

#     dataset[i]["predictions"] = all_summaries


In [5]:
from transformers.pipelines.pt_utils import KeyDataset
from tqdm import tqdm
import numpy as np
import networkx as nx
import networkx.utils

cefr_labels = ["A1", "A2", "B1", "B2", "C1", "C2"]

if not hasattr(np, 'int'):
    np.int = int
    
# 1. Flatten the dataset into "Task Rows"
# Each row in 'flat_data' will represent ONE summary task
flat_data = []
for item in dataset:
    for label in cefr_labels:
        # Pre-render the messages
        msg = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": jinja_template.render(text=item["text"], level=label)}
        ]
        flat_data.append({"messages": msg})

# 2. Use a Generator to stream data (prevents the 'sequential' warning)
def data_generator():
    for item in flat_data:
        # print(item)
        yield item["messages"]

# 3. Run the pipeline (Properly configured)
# 'batch_size' goes here, NOT in model_kwargs
summaries = []
for out in tqdm(summarizer(data_generator(), batch_size=32, max_new_tokens=512), total=len(flat_data)):
    # Extract the text content from the pipeline output
    summary_text = out[-1]["generated_text"][-1]["content"]
    
    # Cleaning logic (removing the preamble if exists)
    # parts = summary_text.split("\n\n")
    # cleaned = "\n\n".join(parts[1:]) if len(parts) > 1 else summary_text
    summaries.append(summary_text)

# 4. Re-shape the results back into your original 18,948 dataset rows
for i in range(len(dataset)):
    start_idx = i * 6
    end_idx = start_idx + 6
    # Map the 6 sequential summaries back to their CEFR labels
    dataset[i]["predictions"] = dict(zip(cefr_labels, summaries[start_idx:end_idx]))

100%|██████████| 600/600 [05:01<00:00,  1.99it/s]


In [ ]:
print(dataset[0])

In [7]:
from collections import Counter

output_file = "dataset/predictions/sft/ca_test_data_final_OFFICIAL.jsonl"

with open(output_file, "w") as fp:
    for instance in dataset:
        fp.write(json.dumps(instance).strip() + "\n")

    